# Credit Risk Resampling Techniques

In [16]:
import warnings
warnings.filterwarnings('ignore')


In [17]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter
from sklearn.preprocessing import LabelEncoder, StandardScaler



# Read the CSV and Perform Basic Data Cleaning

In [18]:

columns = [ "loan_amnt", "int_rate", "total_pymnt_inv","total_pymnt","total_rec_prncp","out_prncp_inv","out_prncp","last_pymnt_amnt","total_rec_int","loan_status"]
   

target = ["loan_status"]

In [19]:
# Load the data
file_path = Path('../Resources/LoanStats_2019Q1.csv.zip')
df = pd.read_csv(file_path, skiprows=1)[:-2]
df = df.loc[:, columns].copy()

# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')

# Drop the null rows
df = df.dropna()

# Remove the `Issued` loan status
issued_mask = df['loan_status'] != 'Issued'
df = df.loc[issued_mask]

# convert interest rate to numerical
df['int_rate'] = df['int_rate'].str.replace('%', '')
df['int_rate'] = df['int_rate'].astype('float') / 100


# Convert the target column values to low_risk and high_risk based on their values
x = {'Current': 'low_risk'}   
df = df.replace(x)

x = dict.fromkeys(['Late (31-120 days)', 'Late (16-30 days)', 'Default', 'In Grace Period'], 'high_risk')    
df = df.replace(x)

df.reset_index(inplace=True, drop=True)

df.head()

,loan_amnt,int_rate,total_pymnt_inv,total_pymnt,total_rec_prncp,out_prncp_inv,out_prncp,last_pymnt_amnt,total_rec_int,loan_status
0,35000.0,0.1308,35016.53,35016.528333,35000.00,0.00,0.00,35067.40,16.53,Fully Paid
1,20000.0,0.2250,719.00,719.000000,569.00,19431.00,19431.00,769.00,150.00,low_risk
2,10500.0,0.1719,355.29,355.290000,295.13,10204.87,10204.87,375.35,60.16,low_risk
3,25000.0,0.2000,873.53,873.530000,706.87,24293.13,24293.13,929.09,166.66,low_risk
4,20000.0,0.2000,485.44,485.440000,363.21,19636.79,19636.79,529.88,122.23,low_risk


# Split the Data into Training and Testing

In [20]:
# Create our features
X = df.drop(columns="loan_status")

# Create our target
y = df["loan_status"]

In [21]:
X.describe()

,loan_amnt,int_rate,total_pymnt_inv,total_pymnt,total_rec_prncp,out_prncp_inv,out_prncp,last_pymnt_amnt,total_rec_int
count,96840.000000,96840.000000,96840.000000,96840.000000,96840.000000,96840.000000,96840.000000,96840.000000,96840.000000
mean,16642.454822,0.126793,1271.841045,1272.141466,968.178312,15667.036073,15669.991342,810.178323,303.923245
std,10365.336053,0.048470,2795.864034,2796.006935,2762.369001,10253.892506,10251.742002,2621.588677,278.026566
min,1000.000000,0.060000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,9000.000000,0.081900,407.835000,408.060000,252.480000,7817.340000,7817.340000,269.480000,109.627500
50%,15000.000000,0.118000,726.530000,726.530000,461.450000,13721.910000,13721.910000,416.110000,216.410000
75%,24000.000000,0.155700,1258.670000,1258.670000,814.280000,22455.850000,22456.870000,673.880000,408.030000
max,40000.000000,0.308900,41418.980000,41418.981165,40000.000000,40000.000000,40000.000000,41409.340000,2735.400000


In [22]:
# Check the balance of our target values
y.value_counts()

low_risk       94116
Fully Paid      2157
high_risk        526
Charged Off       41
Name: loan_status, dtype: int64

In [23]:
# Create X_train, X_test, y_train, y_test

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    random_state=1, 
                                                    stratify=y)
X_train.shape

(72630, 9)

## Data Pre-Processing

Scale the training and testing data using the `StandardScaler` from `sklearn`. Remember that when scaling the data, you only scale the features data (`X_train` and `X_testing`).

In [24]:
# Create the StandardScaler instance
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [25]:
# Fit the Standard Scaler with the training data
# When fitting scaling functions, only train on the training dataset
scaler = StandardScaler()

In [26]:
# Scale the training and testing data
X_Scaler = scaler.fit(X_train)
X_train_scaled = X_Scaler.transform(X_train)
X_test_scaled = X_Scaler.transform(X_test)

# Oversampling

In this section, you will compare two oversampling algorithms to determine which algorithm results in the best performance. You will oversample the data using the naive random oversampling algorithm and the SMOTE algorithm. For each algorithm, be sure to complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

### Naive Random Oversampling

In [27]:
# Resample the training data with the RandomOversampler
from imblearn.over_sampling import RandomOverSampler
ROS = RandomOverSampler(random_state=1)
X_resampled, y_resampled = ROS.fit_resample(X_train, y_train)
Counter(y_resampled)

Counter({'low_risk': 70587,
         'Charged Off': 70587,
         'high_risk': 70587,
         'Fully Paid': 70587})

In [28]:
# Train the Logistic Regression model using the resampled data
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)
y_pred = model.predict(X_test_scaled)

In [29]:
# Calculated the balanced accuracy score

from sklearn.metrics import balanced_accuracy_score

balanced_accuracy_score(y_test, y_pred)

0.5807004122572145

In [30]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix

y_pred = model.predict(X_test)
confusion_matrix(y_test, y_pred)

array([[   10,     0,     0,     0],
       [    0,   538,     0,     1],
       [    0,     1,    86,    45],
       [    0,    42,  8326, 15161]])

In [31]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced

print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

Charged Off       1.00      1.00      1.00      1.00      1.00      1.00        10
 Fully Paid       0.93      1.00      1.00      0.96      1.00      1.00       539
  high_risk       0.01      0.65      0.65      0.02      0.65      0.43       132
   low_risk       1.00      0.64      0.93      0.78      0.78      0.58     23529

avg / total       0.99      0.65      0.93      0.78      0.78      0.59     24210



### SMOTE Oversampling

In [32]:
# Resample the training data with SMOTE
from imblearn.over_sampling import SMOTE
X_resampled, y_resampled = SMOTE(random_state=1, sampling_strategy='auto').fit_resample(X_train, y_train)
Counter(y_resampled)

Counter({'low_risk': 70587,
         'Charged Off': 70587,
         'high_risk': 70587,
         'Fully Paid': 70587})

In [33]:
# Train the Logistic Regression model using the resampled data
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [34]:
# Calculated the balanced accuracy score
y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.8372497443450158

In [35]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)

array([[   10,     0,     0,     0],
       [    0,   538,     0,     1],
       [    0,     1,    93,    38],
       [    0,    44,  8278, 15207]])

In [36]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

Charged Off       1.00      1.00      1.00      1.00      1.00      1.00        10
 Fully Paid       0.92      1.00      1.00      0.96      1.00      1.00       539
  high_risk       0.01      0.70      0.66      0.02      0.68      0.46       132
   low_risk       1.00      0.65      0.94      0.78      0.78      0.59     23529

avg / total       0.99      0.65      0.94      0.78      0.78      0.60     24210



# Undersampling

In this section, you will test an undersampling algorithms to determine which algorithm results in the best performance compared to the oversampling algorithms above. You will undersample the data using the Cluster Centroids algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [37]:
# Resample the data using the ClusterCentroids resampler
from imblearn.under_sampling import ClusterCentroids
CC = ClusterCentroids(random_state=1)
X_resampled, y_resampled = CC.fit_resample(X_train, y_train)
Counter(y_resampled)

Counter({'Charged Off': 31, 'Fully Paid': 31, 'high_risk': 31, 'low_risk': 31})

In [38]:
# Train the Logistic Regression model using the resampled data
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [39]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score

balanced_accuracy_score(y_test, y_pred)

0.8372497443450158

In [40]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
y_pred = model.predict(X_test)
confusion_matrix(y_test, y_pred)

array([[   10,     0,     0,     0],
       [    0,   447,     0,    92],
       [    0,     1,    97,    34],
       [    0,    18, 10748, 12763]])

In [41]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

Charged Off       1.00      1.00      1.00      1.00      1.00      1.00        10
 Fully Paid       0.96      0.83      1.00      0.89      0.91      0.81       539
  high_risk       0.01      0.73      0.55      0.02      0.64      0.41       132
   low_risk       0.99      0.54      0.81      0.70      0.66      0.43     23529

avg / total       0.98      0.55      0.82      0.70      0.67      0.44     24210



# Combination (Over and Under) Sampling

In this section, you will test a combination over- and under-sampling algorithm to determine if the algorithm results in the best performance compared to the other sampling algorithms above. You will resample the data using the SMOTEENN algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [42]:
# Resample the training data with SMOTEENN
from imblearn.combine import SMOTEENN

sm = SMOTEENN(random_state=1)
X_resampled, y_resampled = sm.fit_resample(X_train, y_train)
Counter(y_resampled)

Counter({'Charged Off': 70587,
         'Fully Paid': 70394,
         'high_risk': 68041,
         'low_risk': 67623})

In [43]:
# Train the Logistic Regression model using the resampled data
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [44]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score

balanced_accuracy_score(y_test, y_pred)

0.7766497552737786

In [45]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
y_pred = model.predict(X_test)
confusion_matrix(y_test, y_pred)

array([[   10,     0,     0,     0],
       [    0,   539,     0,     0],
       [    0,     2,    98,    32],
       [    0,    44,  8229, 15256]])

In [46]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

Charged Off       1.00      1.00      1.00      1.00      1.00      1.00        10
 Fully Paid       0.92      1.00      1.00      0.96      1.00      1.00       539
  high_risk       0.01      0.74      0.66      0.02      0.70      0.49       132
   low_risk       1.00      0.65      0.95      0.79      0.79      0.60     23529

avg / total       0.99      0.66      0.95      0.79      0.79      0.61     24210

